In [4]:
import os
import inspect
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import compiling, rasp_to_graph
import numpy as np
from tracr.compiler.validating import validate
from typing import Union, TypeVar
from rasp_generator import map_primitives, sampling, utils
import networkx as nx

from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 


def compile_rasp_to_model(sop: rasp.SOp, vocab={1,2,3,4}, max_seq_len=5, compiler_bos="BOS"):
    return compiling.compile_rasp_to_model(
        sop,
        vocab=vocab,
        max_seq_len=max_seq_len,
        compiler_bos=compiler_bos
    )


TEST_INPUT = [1,2,3,4]

# Program Generation

In [14]:
sampler = sampling.ProgramSampler(validate_compilation=True)
errs = sampler.sample()
print(sampler.program([1,2,3,4]))
print()
print("Retries encountered:")
for err in errs:
    print(err)

[0, 1, 2, 3]

Retries encountered:
No SOps of type bool in scope.
No SOps of type None in scope.
No SOps of type None in scope.
No SOps of type None in scope.
No SOps of type None in scope.
No SOps of type None in scope.
No SOps of type bool in scope.


In [11]:
# print program with and without test input
k = 1
utils.print_program(sampler.sops[-k])
print()
print()
utils.print_program(sampler.sops[-k], TEST_INPUT)

select_26 = Select(tokens, tokens, predicate=Comparison.NEQ)
select_28 = Select(tokens, tokens, predicate=Comparison.EQ)
select_25 = Select(tokens, indices, predicate=Comparison.TRUE)
selector_width_27 = SelectorWidth(select_28)    # type: categorical
selector_width_23 = SelectorWidth(select_25)    # type: categorical
aggregate_24 = Aggregate(select_26, selector_width_27)    # type: categorical
select_22 = Select(selector_width_23, aggregate_24, predicate=Comparison.LT)
selector_width_21 = SelectorWidth(select_22)    # type: categorical


select_26 = Select(tokens, tokens, predicate=Comparison.NEQ)
select_28 = Select(tokens, tokens, predicate=Comparison.EQ)
select_25 = Select(tokens, indices, predicate=Comparison.TRUE)
selector_width_27 = SelectorWidth(select_28)    # output: [1, 1, 1, 1]
selector_width_23 = SelectorWidth(select_25)    # output: [4, 4, 4, 4]
aggregate_24 = Aggregate(select_26, selector_width_27)    # output: [1.0, 1.0, 1.0, 1.0]
select_22 = Select(selector_width_23, ag